Preprocess and Load Data

In [1]:
import os
# Find the latest version of spark 3.3 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-07 01:43:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.31MB/s    in 0.2s    

2022-09-07 01:43:53 (6.31 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Netflix-Movies").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://luiwarang-bucket.s3.amazonaws.com/Netflix_Dataset_Movie_Rating.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("Netflix_Dataset_Movie_Rating.csv"), sep=",", header=True, inferSchema=True)


In [5]:
user_data_df.show()

+--------+----+---------+-------+------+
|movie_id|year|     name|user_id|rating|
+--------+----+---------+-------+------+
|       3|1997|Character| 712664|     5|
|       3|1997|Character|1331154|     4|
|       3|1997|Character|2632461|     3|
|       3|1997|Character|  44937|     5|
|       3|1997|Character| 656399|     4|
|       3|1997|Character| 439011|     1|
|       3|1997|Character|1644750|     3|
|       3|1997|Character|2031561|     4|
|       3|1997|Character| 616720|     4|
|       3|1997|Character|2467008|     4|
|       3|1997|Character| 701730|     2|
|       3|1997|Character|1614320|     4|
|       3|1997|Character| 115498|     3|
|       3|1997|Character| 931626|     2|
|       3|1997|Character| 699878|     4|
|       3|1997|Character|1694958|     3|
|       3|1997|Character|  66414|     5|
|       3|1997|Character|2519847|     5|
|       3|1997|Character| 948069|     3|
|       3|1997|Character|  67315|     4|
+--------+----+---------+-------+------+
only showing top

In [6]:
# List dataframe data types
user_data_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [7]:
# Drop null columns
user_data_df =user_data_df.dropna()

In [8]:
# split into training and testing sets
train, test = user_data_df.randomSplit(weights=(0.8, 0.2))

Building Recommendation Model using ALS

In [9]:
# Build the recommendation model using ALS on the training data
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

als = ALS(maxIter=10, regParam=0.1, rank=8, nonnegative=True, coldStartStrategy="drop",\
          userCol='user_id', itemCol='movie_id', ratingCol='rating')
model = als.fit(train)

Making Predictions with ALS Model

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions

DataFrame[movie_id: int, year: int, name: string, user_id: int, rating: int, prediction: float]

In [11]:
# Preview predictions table sorted by user id and rating
predictions.sort("user_id", "rating").show()

+--------+----+--------------------+-------+------+----------+
|movie_id|year|                name|user_id|rating|prediction|
+--------+----+--------------------+-------+------+----------+
|    1843|1994|Beverly Hills Cop...|      6|     2| 3.0680099|
|    2000|1994|Four Weddings and...|      6|     3| 2.9182014|
|     723|1991|           Curly Sue|      6|     3| 2.7456238|
|    3756|2002|       About Schmidt|      6|     3|  2.397502|
|    3198|1991|   The Addams Family|      6|     3| 2.9830575|
|    1971|1995|Under Siege 2: Da...|      6|     3|  3.074465|
|    3925|2003|The Matrix: Reloaded|      6|     3| 3.3818247|
|     494|1983|Monty Python's Th...|      6|     3|  3.285951|
|    3684|1964|          Goldfinger|      6|     3|  3.371311|
|    2112|2003|            Identity|      6|     3| 3.2584069|
|    4227|1997|      The Full Monty|      6|     3| 2.9210052|
|    2340|1990|Joe Versus the Vo...|      6|     3| 2.7086494|
|    2456|1964|A Fistful of Dollars|      6|     4| 3.4

Evaluate the Predictions

In [12]:
# using RMSE to evaluate the model
evaluator = RegressionEvaluator(metricName='rmse', predictionCol='prediction', labelCol='rating')
rmse = evaluator.evaluate(predictions)
print('RMSE: %.4f' % rmse)

RMSE: 0.8634


Making Movie Recommendations to users

In [13]:
# Make recommendations for all users
user_recs = model.recommendForAllUsers(5)

In [14]:
# Print recommendations table schema
user_recs.printSchema()
type(user_recs)

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movie_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



pyspark.sql.dataframe.DataFrame

In [15]:
# Display recommendations for all users
user_recs.show(10, False)

+-------+----------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                               |
+-------+----------------------------------------------------------------------------------------------+
|6      |[{3456, 3.968379}, {2568, 3.8582587}, {2162, 3.856292}, {2782, 3.820723}, {2452, 3.8063202}]  |
|169    |[{3456, 4.657238}, {4427, 4.493266}, {1476, 4.465171}, {3962, 4.4425826}, {2162, 4.4371853}]  |
|183    |[{3456, 4.54572}, {4306, 4.437381}, {4427, 4.4347696}, {32, 4.4334173}, {1476, 4.427165}]     |
|268    |[{1947, 4.934754}, {3456, 4.912211}, {1256, 4.863822}, {2942, 4.8232155}, {3962, 4.810137}]   |
|283    |[{3456, 4.423377}, {1256, 4.3506007}, {1947, 4.3439503}, {1495, 4.2976503}, {2942, 4.2963247}]|
|291    |[{1947, 4.720321}, {3456, 4.578703}, {2548, 4.5640397}, {1256, 4.546108}, {2942, 4.5033293}]  |
|296    |[{3023, 4.149002}, {2300, 4.061247}, {1222, 4.

In [16]:
# Explode user recs dataframe
from pyspark.sql.functions import explode, col
nrecommendations = user_recs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('user_id', col("rec_exp.movie_id"), col("rec_exp.rating"))
nrecommendations.limit(10).show()

+-------+--------+---------+
|user_id|movie_id|   rating|
+-------+--------+---------+
|      6|    3456| 3.968379|
|      6|    2568|3.8582587|
|      6|    2162| 3.856292|
|      6|    2782| 3.820723|
|      6|    2452|3.8063202|
|    169|    3456| 4.657238|
|    169|    4427| 4.493266|
|    169|    1476| 4.465171|
|    169|    3962|4.4425826|
|    169|    2162|4.4371853|
+-------+--------+---------+



In [17]:
# Drop user_id and rating from original dataset
user_data_df_drop = user_data_df.drop(*("user_id", "rating"))
user_data_df_drop = user_data_df_drop.dropDuplicates(["movie_id"])
user_data_df_drop.show()

+--------+----+--------------------+
|movie_id|year|                name|
+--------+----+--------------------+
|       3|1997|           Character|
|       8|2004|What the #$*! Do ...|
|      16|1996|           Screamers|
|      17|2005|           7 Seconds|
|      18|1994|    Immortal Beloved|
|      26|2004|     Never Die Alone|
|      28|2002|     Lilo and Stitch|
|      30|2003|Something's Gotta...|
|      32|2004|ABC Primetime: Me...|
|      33|2000|Aqua Teen Hunger ...|
|      44|1996|      Spitfire Grill|
|      45|1999|     The Love Letter|
|      46|1964|Rudolph the Red-N...|
|      47|1952|The Bad and the B...|
|      48|2001|      Justice League|
|      52|2002|The Weather Under...|
|      55|1995|                Jade|
|      56|2004|           Carandiru|
|      57|1995|         Richard III|
|      58|1996|         Dragonheart|
+--------+----+--------------------+
only showing top 20 rows



In [18]:
# Get movie name from original dataset and join with recommendations table for one user
nrecommendations.join(user_data_df_drop, on='movie_id',how="left").filter('user_id = 6').show()

+--------+-------+---------+----+--------------------+
|movie_id|user_id|   rating|year|                name|
+--------+-------+---------+----+--------------------+
|    3456|      6| 3.968379|2004|      Lost: Season 1|
|    2782|      6| 3.820723|1995|          Braveheart|
|    2568|      6|3.8582587|2004|Stargate SG-1: Se...|
|    2452|      6|3.8063202|2001|Lord of the Rings...|
|    2162|      6| 3.856292|2000|       CSI: Season 1|
+--------+-------+---------+----+--------------------+



In [19]:
# Get movie name from original dataset and join with recommendations table for all users
final_recs = nrecommendations.join(user_data_df_drop, on='movie_id',how="left")

In [20]:
# Convert datfarame to pandas
from google.colab import files
final_recs_pan = final_recs.toPandas()

In [21]:
# Convert datfarame to csv
final_recs_pan.to_csv("final_recs_pan.csv")

In [22]:
# Export csv file to local pc
files.download('final_recs_pan.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>